In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/641.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/173.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/815.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/491.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/718.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/709.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/379.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/780.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/248.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/94.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/480.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/236.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/771.png
/kaggle/input/image-super-resolution/dataset/Raw Data/low_res/675.png
/kaggle/input/image-s

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import matplotlib.pyplot as plt

# Residual Unit with two Conv layers, residual connection, and Batch Normalization
def residual_unit(input_tensor, filters, kernel_size=3):
    x = layers.Conv2D(filters, (kernel_size, kernel_size), padding='same', kernel_initializer='he_normal')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, (kernel_size, kernel_size), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    return layers.Add()([input_tensor, x])  # Local residual connection

# Recursive Block (with shared weights among residual units)
def recursive_block(input_tensor, num_residual_units, filters):
    x = input_tensor
    for _ in range(num_residual_units):
        x = residual_unit(x, filters)
    return x

# DRRN Model
def DRRN(input_shape, num_recursive_blocks, num_residual_units, filters):
    inputs = Input(shape=input_shape)

    # Initial Conv Layer
    x = layers.Conv2D(filters, (3, 3), padding='same')(inputs)

    # Recursive blocks with residual connections
    for _ in range(num_recursive_blocks):
        x = recursive_block(x, num_residual_units, filters)

    # Output Conv Layer to reconstruct residual image
    x = layers.Conv2D(3, (3, 3), padding='same')(x)

    # Global Residual Learning (add the input LR image to the predicted residual)
    outputs = layers.Add()([inputs, x])

    # Build model
    model = models.Model(inputs, outputs)
    return model

# Function to extract 31x31 patches with a stride of 21 from a single image
def extract_patches(image, patch_size=31, stride=21):
    patches = []
    h, w, _ = image.shape
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y + patch_size, x:x + patch_size]
            patches.append(patch)
    return np.array(patches)

# Function to load images from a directory and extract low-res and high-res patches
def load_and_extract_patches(low_res_dir, high_res_dir, patch_size=31, stride=21):
    low_res_patches = []
    high_res_patches = []

    # Assuming filenames are identical for both low-res and high-res images
    for file_name in os.listdir(low_res_dir):
        # Load low-resolution image
        low_res_image_path = os.path.join(low_res_dir, file_name)
        low_res_image = cv2.imread(low_res_image_path)
        
        # Load high-resolution image
        high_res_image_path = os.path.join(high_res_dir, file_name)
        high_res_image = cv2.imread(high_res_image_path)
        
        # Ensure both images have at least patch_size dimensions and are the same size
        if (low_res_image.shape[0] >= patch_size and low_res_image.shape[1] >= patch_size and
            high_res_image.shape == low_res_image.shape):
            
            # Extract patches from both low-res and high-res images
            low_res_patches.extend(extract_patches(low_res_image, patch_size, stride))
            high_res_patches.extend(extract_patches(high_res_image, patch_size, stride))
    
    return np.array(low_res_patches), np.array(high_res_patches)

# Function to train and evaluate the model with different numbers of layers
def train_and_evaluate_drrn(num_residual_units_list, num_recursive_blocks, filters, input_shape, X_train, Y_train, epochs=5):
    train_loss_histories = []
    val_loss_histories = []
    
    for num_residual_units in num_residual_units_list:
        print(f"\nTraining DRRN model with {num_residual_units} residual units in each recurrent block...\n")
        
        # Build the DRRN model
        drrn_model = DRRN(input_shape, num_recursive_blocks, num_residual_units, filters)
        
        # Compile the model with a lower learning rate and gradient clipping
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipvalue=1.0)
        drrn_model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
        
        # Train the model
        history = drrn_model.fit(X_train, Y_train, batch_size=128, epochs=epochs, validation_split=0.1, verbose=1)
        
        # Save the loss history for plotting
        train_loss_histories.append(history.history['loss'])
        val_loss_histories.append(history.history['val_loss'])
        
        # Save the model for future use (optional)
        drrn_model.save(f"drrn_model_{num_residual_units}_residual_units.h5")
    
    return train_loss_histories, val_loss_histories

# Plotting function to visualize the effect of different numbers of residual units on log scale
def plot_loss_histories(num_residual_units_list, train_loss_histories, val_loss_histories):
    plt.figure(figsize=(12, 6))
    
    for i, num_residual_units in enumerate(num_residual_units_list):
        plt.plot(np.log10(train_loss_histories[i]), label=f'Train Loss (Residual Units: {num_residual_units})')
        plt.plot(np.log10(val_loss_histories[i]), label=f'Val Loss (Residual Units: {num_residual_units})')
    
    plt.title('Effect of Residual Units on Training and Validation Loss (Log Scale)')
    plt.xlabel('Epochs')
    plt.ylabel('Log Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Define the paths to your low-resolution and high-resolution images
low_res_dir = "/kaggle/input/image-super-resolution/dataset/Raw Data/low_res"
high_res_dir = "/kaggle/input/image-super-resolution/dataset/Raw Data/high_res"

# Extract low-res and high-res patches
X_train, Y_train = load_and_extract_patches(low_res_dir, high_res_dir, patch_size=31, stride=21)

# Normalize the patches (if necessary)
X_train = X_train / 255.0  # Normalize low-res patches (input)
Y_train = Y_train / 255.0  # Normalize high-res patches (target)

# Reshape the data for model input
X_train = X_train.reshape(-1, 31, 31, 3)  # Low-res patches
Y_train = Y_train.reshape(-1, 31, 31, 3)  # High-res patches

print(f"Extracted {X_train.shape[0]} patches of size 31x31 from the dataset.")

# Define the input shape (LR image), number of recursive blocks, and filters
input_shape = (31, 31, 3)  # (height, width, channels)
num_recursive_blocks = 3  # Number of recursive blocks
filters = 64              # Number of filters in Conv layers

# Define the number of residual units (layers) to test in the recurrent block
num_residual_units_list = [4, 6, 8, 10, 12]  

# Train and evaluate the model for different numbers of residual units
train_loss_histories, val_loss_histories = train_and_evaluate_drrn(
    num_residual_units_list, num_recursive_blocks, filters, input_shape, X_train, Y_train, epochs=15
)

# Plot the results
plot_loss_histories(num_residual_units_list, train_loss_histories, val_loss_histories)


Extracted 103455 patches of size 31x31 from the dataset.

Training DRRN model with 4 residual units in each recurrent block...

Epoch 1/15


I0000 00:00:1727759095.262435     100 service.cc:145] XLA service 0x79d4d406e250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727759095.262501     100 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/728 ━━━━━━━━━━━━━━━━━━━━ 4:23:08 22s/step - accuracy: 0.3800 - loss: 20.6838

I0000 00:00:1727759102.914446     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


728/728 ━━━━━━━━━━━━━━━━━━━━ 98s 105ms/step - accuracy: 0.3979 - loss: 1.2633 - val_accuracy: 0.4924 - val_loss: 0.0629
Epoch 2/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 94ms/step - accuracy: 0.4767 - loss: 0.0433 - val_accuracy: 0.5463 - val_loss: 0.0233
Epoch 3/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 94ms/step - accuracy: 0.5167 - loss: 0.0219 - val_accuracy: 0.5602 - val_loss: 0.0172
Epoch 4/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 94ms/step - accuracy: 0.5464 - loss: 0.0146 - val_accuracy: 0.6297 - val_loss: 0.0105
Epoch 5/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 94ms/step - accuracy: 0.5733 - loss: 0.0106 - val_accuracy: 0.5674 - val_loss: 0.0081
Epoch 6/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 95ms/step - accuracy: 0.5972 - loss: 0.0080 - val_accuracy: 0.6458 - val_loss: 0.0058
Epoch 7/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 95ms/step - accuracy: 0.6095 - loss: 0.0070 - val_accuracy: 0.6137 - val_loss: 0.0091
Epoch 8/15
728/728 ━━━━━━━━━━━━━━━━━━━━ 69s 94ms/step - accuracy: 0.6246 - loss: 0.0061 - val_accura